## Zero-Shot Image Classification

We use Hugging Face’s `pipeline` to load a CLIP model for zero-shot image classification

In [1]:
from transformers import pipeline

checkpoint = "openai/clip-vit-large-patch14"
detector = pipeline(model=checkpoint, task="zero-shot-image-classification")
#checkpoint = "google/siglip-so400m-patch14-384"
#detector = pipeline(task="zero-shot-image-classification", model="google/siglip-so400m-patch14-384")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


## Loading the Fashion-MNIST Dataset

We load Zalando’s Fashion-MNIST from the Hugging Face Hub using the `datasets` library

In [2]:
from datasets import load_dataset

dataset = load_dataset('zalando-datasets/fashion_mnist')
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

## Displaying a Single Example

You can render the raw PIL image for the first training sample directly

In [3]:
dataset['train'][0]['image']

## Defining Candidate Labels

We list the 10 Fashion-MNIST classes as strings for CLIP’s zero-shot pipeline, and prepare containers to hold true vs. predicted labels

In [4]:
from PIL import Image
import io
from tqdm import tqdm

labels_fashion_mnist= [
    'T - shirt / top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot'
]

true_labels = []
predicted_labels = []

In [ ]:
from tqdm import tqdm
from PIL import Image
import io

true_labels = []
predicted_labels = []

for example in tqdm(dataset['train']):
    # image ist bereits ein PIL-Image
    image = example['image']


    # Zero-Shot-Vorhersage
    results = detector(image, candidate_labels=labels_fashion_mnist)
    sorted_results = sorted(results, key=lambda x: x["score"], reverse=True)
    pred_label = sorted_results[0]["label"]

    true_labels.append(example["label"])
    predicted_labels.append(pred_label)


100%|██████████| 60000/60000 [36:17<00:00, 27.55it/s]


## Evaluating Zero-Shot Classification Performance

We convert the ground-truth integer labels into their string names, then use `sklearn.metrics` to compute accuracy, precision, and recall (weighted over all 10 classes).

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score


# 1) True-Labels von Integer → String konvertieren
true_labels_str = [labels_fashion_mnist[i] for i in true_labels]

# 2) Accuracy berechnen
accuracy = accuracy_score(true_labels_str, predicted_labels)

# 3) Precision und Recall berechnen (nur Strings in `labels=` übergeben)
precision = precision_score(
    true_labels_str,
    predicted_labels,
    average='weighted',
    labels=labels_fashion_mnist
)
recall = recall_score(
    true_labels_str,
    predicted_labels,
    average='weighted',
    labels=labels_fashion_mnist
)

# 4) Ergebnisse ausgeben
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")


Accuracy : 0.6097
Precision: 0.5647
Recall   : 0.6097


## Gradio example

In [7]:
import gradio as gr
from transformers import pipeline

# 1) Models laden
vit_classifier = pipeline(
    task="image-classification",
    model="Granitagushi/vit-base-fashion"
)
clip_detector = pipeline(
    task="zero-shot-image-classification",
    model="openai/clip-vit-large-patch14"
)

labels = [
    'T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# 2) Callback mit str-Casting
def classify_clothing(image):
    vit_results  = vit_classifier(image)
    vit_output   = { str(r["label"]): float(r["score"]) for r in vit_results }

    clip_results = clip_detector(image, candidate_labels=labels)
    clip_output  = { str(r["label"]): float(r["score"]) for r in clip_results }

    return {
        "ViT Classification": vit_output,
        "CLIP Zero-Shot Classification": clip_output
    }

# 3) Beispiele nur, wenn die Dateien ins Repo wandern
example_images = [
    ["example_images/jeans.jpg"],
    ["example_images/kleid.jpg"],
    ["example_images/pullover.jpg"],
    ["example_images/sandale.jpg"],
    ["example_images/tasche.jpg"]
]

# 4) Interface – cache_examples im Konstruktor
iface = gr.Interface(
    fn=classify_clothing,
    inputs=gr.Image(type="pil", label="Upload ein Fashion-MNIST-Bild"),
    outputs=gr.JSON(label="Ergebnisse"),
    title="Fashion MNIST Klassifikation",
    description="Vergleiche dein ViT-Modell mit einem CLIP Zero-Shot-Modell.",
    examples=example_images,    # <— nur, wenn Dateien wirklich da sind
    cache_examples=False
)

if __name__ == "__main__":
    iface.launch(share=True)

config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

Device set to use cuda:0
Device set to use cuda:0


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://f20bea4282b020af30.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
